In [ ]:
%load_ext jupyflink.magics

In [ ]:
%flink_gateway_init --debug

In [ ]:
%%flink_gateway_sql_prepare
ADD JAR '/opt/gateway/jars/flink-sql-connector-kafka-1.16.2.jar';

In [ ]:
%%flink_gateway_sql_prepare
drop table if exists attributes;
CREATE TABLE attributes (
  `id` STRING,
  `entityId` STRING,
  `name` STRING,
  `nodeType` STRING,
  `valueType` STRING,
  `index` INTEGER,
  `type` STRING,
  `https://uri.etsi.org/ngsi-ld/datasetId` STRING,
  `https://uri.etsi.org/ngsi-ld/hasValue` STRING,
  `https://uri.etsi.org/ngsi-ld/hasObject` STRING,
  `ts` TIMESTAMP(3) METADATA FROM 'timestamp',
  WATERMARK FOR ts AS ts
) WITH (
  'connector' = 'kafka',
  'topic' = 'iff.ngsild.attributes',
  'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
);
DROP VIEW IF EXISTS `attributes_view`;
CREATE VIEW `attributes_view` AS
SELECT `id`,`type`,
`entityId`,
`name`,
`nodeType`,
`valueType`,
`index`,
`https://uri.etsi.org/ngsi-ld/datasetId`,
`https://uri.etsi.org/ngsi-ld/hasValue`,
`https://uri.etsi.org/ngsi-ld/hasObject`,
`ts` FROM (
  SELECT *,
ROW_NUMBER() OVER (PARTITION BY `id`,`https://uri.etsi.org/ngsi-ld/datasetId`
ORDER BY ts DESC) AS rownum
FROM `attributes` )
WHERE rownum = 1;


In [ ]:
%%flink_gateway_sql_prepare
drop table if exists ngsild_updates;
CREATE TABLE ngsild_updates (
  `op` STRING,
  `overwriteOrReplace` Boolean,
  `noForward` Boolean,
  `entities` STRING
) WITH (
  'connector' = 'kafka',
  'topic' = 'iff.ngsild-updates',
  'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
  'scan.startup.mode' = 'latest-offset',
  'format' = 'json'
);